# Define evaluation logic

In [33]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [1]:
import time
import json1
import pickle
import re
import html
import sys
import gc

import gensim

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, scale
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

import pandas as pd
from pandas import DataFrame

from scipy.stats import spearmanr, pearsonr
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from nltk import word_tokenize
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize.casual import TweetTokenizer

from IPython.display import display, HTML

Using TensorFlow backend.


In [2]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("Jupyter Notification",{icon:"http://blog.jupyter.org/content/images/2015/02/jupyter-sq-text.png",body:"' + message + '"});</script>'))

In [71]:
glove_twitter_path = "/Users/COMMANDER/Desktop/MScBA Consultantcy Project/GloVe/glove.twitter.27B/"
glove_path_6B = "/Users/COMMANDER/Desktop/MScBA Consultantcy Project/GloVe/glove.6B/"
glove_path_42B = "/Users/COMMANDER/Desktop/MScBA Consultantcy Project/GloVe/glove.42B.300d/"
glove_path_840B = "/Users/COMMANDER/Desktop/MScBA Consultantcy Project/GloVe/glove.840B.300d/"
glove_path_twitter = "/Users/COMMANDER/Desktop/MScBA Consultantcy Project/GloVe/glove.twitter.27B/"
wassa_home = "/Users/COMMANDER/Downloads/"

## Word2Vec + GloVe

In [47]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    model = {}
    num = 1
    for line in f:
        try:
            splitLine = line.split()
            word = splitLine[0]
            embedding = [float(val) for val in splitLine[1:]]
            model[word] = np.array(embedding)
            num += 1
        except Exception as e:
            print("Failed at line " + str(num))
    print("Done.",len(model)," words loaded!")
    return model

In [54]:
# Google news pretrained vectors
wv_model_path = "GoogleNews-vectors-negative300.bin.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path, binary=True, unicode_errors='ignore')

In [6]:
# Twitter pretrained vectors
wv_model_path_1 = "word2vec_twitter_model.bin"
wv_model_1 = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path_1, binary=True, unicode_errors='ignore')

In [41]:
wv_model_path_2 = glove_path_twitter + "glove.twitter.27B.200d.txt"
wv_model_2 = loadGloveModel(wv_model_path_2)

Loading Glove Model
Done. 1193514  words loaded!


In [42]:
wv_model_path_3 = glove_path_6B + "glove.6B.300d.txt"
wv_model_3 = loadGloveModel(wv_model_path_3)


Loading Glove Model
Done. 400000  words loaded!


In [45]:
wv_model_path_4 = glove_path_42B + "glove.42B.300d.txt"
wv_model_4 = loadGloveModel(wv_model_path_4)

Loading Glove Model
Done. 1917494  words loaded!


In [49]:
wv_model_path_5 = glove_path_840B + "glove.840B.300d.txt"
wv_model_5 = loadGloveModel(wv_model_path_5)

Loading Glove Model
Failed at line 52344
Failed at line 128261
Failed at line 151101
Failed at line 200666
Failed at line 209830
Failed at line 220775
Failed at line 253456
Failed at line 365739
Failed at line 532041
Failed at line 717294
Failed at line 994809
Failed at line 1123321
Failed at line 1148398
Failed at line 1352098
Failed at line 1499714
Failed at line 1533795
Failed at line 1899826
Failed at line 1921136
Failed at line 2058949
Failed at line 2165228
Done. 2195885  words loaded!


In [56]:
w2v_dimensions = len(wv_model['word'])
w2v_dimensions_1 = len(wv_model_1['word'])
w2v_dimensions_2 = len(wv_model_2['word'])
w2v_dimensions_3 = len(wv_model_3['word'])
w2v_dimensions_4 = len(wv_model_4['word'])
w2v_dimensions_5 = len(wv_model_5['word'])

print(w2v_dimensions, w2v_dimensions_1, w2v_dimensions_2, w2v_dimensions_3, w2v_dimensions_4, w2v_dimensions_5)

300 400 200 300 300 300


In [57]:
def get_word2vec_embedding(word, model, dimensions):

    vec_rep = np.zeros(dimensions)
    if word in model:
        vec_rep = model[word]
    
    return vec_rep

In [62]:
get_word2vec_embedding("charm", wv_model_1, 200)

array([ 0.19077739, -0.02349201,  0.3269487 ,  0.3995059 , -0.06090572,
       -0.5030889 , -0.03499613, -0.18743885, -0.14450862, -0.10861609,
       -0.13043003, -0.16187936,  0.2602683 , -0.37468734,  0.34821877,
        0.46345547,  0.2048301 , -0.3581504 , -0.0781073 , -0.3599439 ,
       -0.5026346 , -0.01241032, -0.02729836, -0.0931119 , -0.11458384,
       -0.07938592, -0.00769653,  0.30210668,  0.32244754,  0.10260496,
       -0.23955123, -0.08493695,  0.07506456,  0.16352107,  0.27562946,
       -0.19905256,  0.5133967 ,  0.48387128,  0.15543362,  0.32026324,
        0.00181202,  0.48273942, -0.06955759, -0.38079122, -0.0769409 ,
       -0.1887485 ,  0.04991629,  0.03879244,  0.5918345 ,  0.14987682,
        0.08685969, -0.2047459 ,  0.33729124,  0.17343895,  0.20181322,
        0.32459497,  0.07476694,  0.00492099, -0.40399665, -0.4677339 ,
       -0.49484017,  0.31548923,  0.25549474,  0.04917512,  0.05127952,
        0.50814366, -0.07589386, -0.43660858, -0.30149156, -0.03

# Data Cleaning

In [63]:
wnl = WordNetLemmatizer()
tknzr = TweetTokenizer()

In [64]:
def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)

In [241]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [65]:
def clean_str(string):  
    string = html.unescape(string)
    string = string.replace("\\n", " ")
    string = string.replace("_NEG", "")
    string = string.replace("_NEGFIRST", "")
    string = re.sub(r"@[A-Za-z0-9_(),!?\'\`]+", " ", string) # removing any twitter handle mentions
    string = re.sub(r"\d+", " ", string) # removing any words with numbers
    string = re.sub(r"_", " ", string)
    string = re.sub(r":", " ", string)
    string = re.sub(r"/", " ", string)
    string = re.sub(r"#", " ", string)
    string = re.sub(r"\.", " ", string)
    string = re.sub(r"\*", " ", string)
    string = re.sub(r"\'s", " ", string)
    string = re.sub(r"\'m", " am", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"n\’t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\’re", " are", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\’d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"\’ll", " will", string)
    string = re.sub(r"'", " ", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", " !", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", " ?", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"<", " ", string)
    string = re.sub(r">", " ", string)
    string = re.sub(r";", " ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return remove_stopwords(string.strip().lower())

# Metadata and Class Definitions

In [72]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [165]:
def read_training_data(training_data_file_path):

    train_list = list()
    with open(training_data_file_path, encoding="utf8") as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], list(tknzr.tokenize(clean_str(array[1]))), 
                                    array[2], float(array[3])))
    return train_list
            
def read_training_data_verbatim(training_data_file_path):

    train_list = list()
    with open(training_data_file_path, encoding="utf8") as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], array[1], array[2], float(array[3])))
    return train_list
    
def read_test_data(training_data_file_path):

    test_list = list()
    with open(training_data_file_path, encoding="utf8") as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            test_list.append(Tweet(array[0], clean_str(array[1]), array[2], None))
    return test_list

In [74]:
non_linear_factor = PolynomialFeatures(3)

In [69]:
emotion = "anger"

In [242]:
training_data_file_path = \
    wassa_home + "corpora/" + \
    emotion + "-ratings-0to1.train.txt"
predictions_file_path = \
    wassa_home + "predictions/" + \
    emotion + "-pred.txt"
dev_set_path = \
    wassa_home + "corpora/" + \
    emotion + "-ratings-0to1.dev.gold.txt"
test_data_file_path = \
    wassa_home + "corpora/" + \
    emotion + "-ratings-0to1.test.gold.txt"
debug_file_path = \
    wassa_home + "dataset/test-set/debug/" + \
    emotion + ".tsv"
word_embeddings_path = wassa_home + "embeddings/" + emotion + "-word-embeddings.pkl"

# Feature Extraction Snippets

## Emoji Intensity (Thanks to Roger, it works!)

In [182]:
with open('/Users/COMMANDER/Desktop/index.json', encoding="utf8") as emoji_file:
    emoji_list = json.load(emoji_file)
    
emoji_dict = dict()

for emoji in emoji_list:
    emoji_dict[emoji["emoji"]] = (emoji["name"], emoji["polarity"])

In [188]:
emoji_list

[{'name': '100', 'emoji': '💯', 'polarity': 3},
 {'name': 'angry', 'emoji': '😠', 'polarity': -3},
 {'name': 'anguished', 'emoji': '😧', 'polarity': -3},
 {'name': 'astonished', 'emoji': '😲', 'polarity': 2},
 {'name': 'black_heart', 'emoji': '🖤', 'polarity': 3},
 {'name': 'blue_heart', 'emoji': '💙', 'polarity': 3},
 {'name': 'blush', 'emoji': '😊', 'polarity': 2},
 {'name': 'broken_heart', 'emoji': '💔', 'polarity': -3},
 {'name': 'clap', 'emoji': '👏', 'polarity': 3},
 {'name': 'clown_face', 'emoji': '🤡', 'polarity': 0},
 {'name': 'cold_sweat', 'emoji': '😰', 'polarity': -2},
 {'name': 'confounded', 'emoji': '😖', 'polarity': -2},
 {'name': 'confused', 'emoji': '😕', 'polarity': -2},
 {'name': 'cowboy_hat_face', 'emoji': '🤠', 'polarity': 2},
 {'name': 'crossed_fingers', 'emoji': '🤞', 'polarity': 2},
 {'name': 'cry', 'emoji': '😢', 'polarity': -2},
 {'name': 'crying_cat_face', 'emoji': '😿', 'polarity': -2},
 {'name': 'cupid', 'emoji': '💘', 'polarity': 3},
 {'name': 'disappointed', 'emoji': '😞', 

In [189]:
def get_emoji_intensity(word):
    
    score = 0.0
    if word in emoji_dict.keys():
        score = float(emoji_dict[word][1])
    
    vec_rep = np.array([score])
    
    
    return non_linear_factor.fit_transform([vec_rep])[0]

In [191]:
get_emoji_intensity("😠")

array([  1.,  -3.,   9., -27.])

## Emotion Intensity Lexicon

In [99]:
affect_intensity_file_path = \
    wassa_home + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"
    
def get_word_affect_intensity_dict(emotion):
    word_intensities = dict()

    with open(affect_intensity_file_path) as affect_intensity_file:
        for line in affect_intensity_file:
            word_int_array = line.replace("\n", "").split("\t")

            if (word_int_array[2] == emotion):
                word_intensities[word_int_array[0]] = float(word_int_array[1])

    return word_intensities

In [243]:
word_intensities = get_word_affect_intensity_dict(emotion)

In [244]:
word_intensities 

{'outraged': 0.964,
 'brutality': 0.959,
 'hatred': 0.953,
 'hateful': 0.94,
 'terrorize': 0.939,
 'infuriated': 0.938,
 'violently': 0.938,
 'furious': 0.929,
 'enraged': 0.927,
 'furiously': 0.927,
 'screwyou': 0.924,
 'murderer': 0.922,
 'fury': 0.922,
 'execution': 0.917,
 'angered': 0.916,
 'savagery': 0.915,
 'slaughtering': 0.914,
 'veryangry': 0.913,
 'assassinate': 0.912,
 'annihilation': 0.912,
 'fuckoff': 0.912,
 'rage': 0.911,
 'loathe': 0.909,
 'damnation': 0.906,
 'fucktard': 0.906,
 'homicidal': 0.906,
 'roadrage': 0.906,
 'furor': 0.9,
 'hostile': 0.898,
 'annihilate': 0.898,
 'murder': 0.897,
 'raging': 0.896,
 'explosive': 0.894,
 'infuriates': 0.894,
 'pissed': 0.894,
 'ferocious': 0.894,
 'obliterated': 0.894,
 'rape': 0.894,
 'vengeful': 0.894,
 'sopissed': 0.894,
 'killing': 0.893,
 'combative': 0.891,
 'gofuckyourself': 0.886,
 'vengeance': 0.886,
 'wrath': 0.885,
 'torment': 0.885,
 'vicious': 0.884,
 'massacre': 0.882,
 'threatening': 0.882,
 'abhorrent': 0.875

In [101]:
def get_emo_int_vector(word):
    
    score = 0.0
    if word in word_intensities.keys():
        score = float(word_intensities[word])
        
    vec_rep = np.array([score])
    
    return non_linear_factor.fit_transform([vec_rep])[0]

In [102]:
get_emo_int_vector('fury')

array([1.        , 0.922     , 0.850084  , 0.78377745])

## SentiWordNet 

In [103]:
def get_sentiwordnetscore(word):
    
    vec_rep = np.zeros(2)
    
    synsetlist = list(swn.senti_synsets(word))

    if synsetlist:
        vec_rep[0] = synsetlist[0].pos_score()
        vec_rep[1] = synsetlist[0].neg_score()

    return non_linear_factor.fit_transform([vec_rep])[0]

In [104]:
get_sentiwordnetscore("fury")

array([1.      , 0.25    , 0.5     , 0.0625  , 0.125   , 0.25    ,
       0.015625, 0.03125 , 0.0625  , 0.125   ])

## Sentiment Emotion Presence Lexicon

In [105]:
sentiment_emotion_lex_file_path = \
    wassa_home + \
    "Lexikon/NRC-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

def get_affect_presence_list(emotion):
    word_list = list()
    
    with open(sentiment_emotion_lex_file_path) as sentiment_emotion_lex_file:
        for line in sentiment_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")


            if (word_array[1] == emotion and word_array[2] == '1'):
                word_list.append(word_array[0])
    return word_list            

In [106]:
sentiment_emotion_lex_word_list = get_affect_presence_list(emotion)

In [245]:
sentiment_emotion_lex_word_list

['abandoned',
 'abandonment',
 'abhor',
 'abhorrent',
 'abolish',
 'abomination',
 'abuse',
 'accursed',
 'accusation',
 'accused',
 'accuser',
 'accusing',
 'actionable',
 'adder',
 'adversary',
 'adverse',
 'adversity',
 'advocacy',
 'affront',
 'aftermath',
 'aggravated',
 'aggravating',
 'aggravation',
 'aggression',
 'aggressive',
 'aggressor',
 'agitated',
 'agitation',
 'agony',
 'alcoholism',
 'alienate',
 'alienation',
 'allegation',
 'altercation',
 'ambush',
 'anarchism',
 'anarchist',
 'anarchy',
 'anathema',
 'anger',
 'angry',
 'anguish',
 'animosity',
 'animus',
 'annihilate',
 'annihilated',
 'annihilation',
 'annoy',
 'annoyance',
 'annoying',
 'antagonism',
 'antagonist',
 'antagonistic',
 'antichrist',
 'antipathy',
 'antisocial',
 'antithesis',
 'anxiety',
 'argue',
 'argument',
 'argumentation',
 'arguments',
 'armament',
 'armed',
 'arraignment',
 'arrogant',
 'arson',
 'assail',
 'assailant',
 'assassin',
 'assassinate',
 'assassination',
 'assault',
 'asshole',


In [107]:
def get_sentiment_emotion_feature(word):
    
    score = 0.0
    if word in sentiment_emotion_lex_word_list:
        score = 1.0
    vec_rep = np.array([score])
    
    return non_linear_factor.fit_transform([vec_rep])[0]

In [108]:
get_sentiment_emotion_feature("fury")

array([1., 1., 1., 1.])

## Hashtag Emotion Intensity

In [109]:
hashtag_emotion_lex_file_path = \
    wassa_home + \
    "Lexikon/NRC-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/" + \
    "NRC-Hashtag-Emotion-Lexicon-v0.2.txt"
    
def get_hashtag_emotion_intensity(emotion):
    hastag_intensities = dict()
    
    with open(hashtag_emotion_lex_file_path) as hashtag_emotion_lex_file:
        for line in hashtag_emotion_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if (word_array[0] == emotion):
                hastag_intensities[word_array[1]] = float(word_array[2])
                
    return hastag_intensities

In [110]:
hashtag_emotion_intensities = get_hashtag_emotion_intensity(emotion)

In [246]:
hashtag_emotion_intensities

{'#agitated': 1.42991541495012,
 '#roar': 1.42991541495012,
 'revving': 1.42991541495012,
 '#backgroundcheck': 1.42991541495012,
 '#hormonal': 1.42991541495012,
 '#hangry': 1.42991541495012,
 'darkside': 1.42991541495012,
 '#imangry': 1.42991541495012,
 '#point': 1.42991541495012,
 'times&lt': 1.42991541495012,
 '#pistoff': 1.42991541495012,
 'dents': 1.42991541495012,
 'mts': 1.42991541495012,
 '#vexed': 1.42991541495012,
 '#infuriated': 1.42991541495012,
 '#blackwidow': 1.42991541495012,
 '#fury': 1.42991541495012,
 '#fuckedoff': 1.42991541495012,
 '#veryangry': 1.42991541495012,
 '#ragemode': 1.42991541495012,
 'bluddy': 1.42991541495012,
 '#overplayed': 1.42991541495012,
 '#sogay': 1.42991541495012,
 'pantera': 1.42991541495012,
 '#fiesty': 1.42991541495012,
 '#getyourown': 1.42991541495012,
 '#wokemeup': 1.42991541495012,
 'asleeep': 1.42991541495012,
 '#capslock': 1.42991541495012,
 '#aggressive': 1.42991541495012,
 'tamping': 1.42991541495012,
 '#dontmess': 1.42991541495012,
 'g

In [111]:
def get_hashtag_emotion_vector(word):
    
    score = 0.0
    
    if word in hashtag_emotion_intensities.keys():
        score = float(hashtag_emotion_intensities[word])
        
    vec_rep = np.array([score])
            
    return non_linear_factor.fit_transform([vec_rep])[0]

In [112]:
get_hashtag_emotion_vector("#fury")

array([1.        , 1.42991541, 2.04465809, 2.92368813])

## Emoticon Sentiment Lexicon

In [115]:
emoticon_lexicon_unigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-unigrams.txt"
emoticon_lexicon_bigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-bigrams.txt"
    
emoticon_lexicon_unigrams = dict()
emoticon_lexicon_bigrams = dict()

def get_emoticon_lexicon_unigram_dict():
    with open(emoticon_lexicon_unigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_lexicon_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_lexicon_unigrams

def get_emoticon_lexicon_bigram_dict():
    with open(emoticon_lexicon_bigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_lexicon_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_lexicon_bigrams

In [116]:
emoticon_lexicon_unigram_dict = get_emoticon_lexicon_unigram_dict()

In [247]:
emoticon_lexicon_unigram_dict

{'@jeffrey_donovan': array([5., 6., 0.]),
 'familar': array([5., 6., 0.]),
 '@vppatel2011': array([5., 6., 0.]),
 'emilio': array([5., 7., 0.]),
 '@livetolovemcfly': array([ 5., 11.,  0.]),
 '@j2ad': array([5., 6., 0.]),
 'http://twitpic.com/65p32': array([5., 5., 0.]),
 '@missashleyn': array([5., 5., 0.]),
 'http://twitpic.com/6fauf': array([5., 6., 0.]),
 'on-site': array([5., 6., 0.]),
 '@daihard': array([5., 6., 0.]),
 '@banksyart': array([  5., 158.,   0.]),
 'supertramp': array([5., 7., 0.]),
 'spinvox': array([5., 5., 0.]),
 '@thej': array([5., 8., 0.]),
 '@d_hock': array([5., 5., 0.]),
 '@tprettyman': array([5., 6., 0.]),
 'studmuffin': array([5., 5., 0.]),
 '@fourstringfuror': array([5., 9., 0.]),
 '@sharongerlach': array([5., 6., 0.]),
 '@leesaamarie': array([5., 6., 0.]),
 '@goldigold': array([5., 5., 0.]),
 '@ktjlarsen': array([5., 6., 0.]),
 '@lar103081': array([5., 5., 0.]),
 'bessst': array([5., 7., 0.]),
 '@anannyadeb': array([5., 5., 0.]),
 '@divinechihuahua': array([5

In [117]:
emoticon_lexicon_bigram_dict = get_emoticon_lexicon_bigram_dict()

In [248]:
emoticon_lexicon_bigram_dict

{'add everyone': array([7.352e+00, 1.485e+03, 1.000e+00]),
 'or pay': array([   5.975, 1499.   ,    4.   ]),
 'train or': array([   5.561, 1487.   ,    6.   ]),
 'why limit': array([  5.55, 245.  ,   1.  ]),
 '140 ch': array([  5.474, 227.   ,   1.   ]),
 'everyone you': array([   5.284, 1502.   ,    8.   ]),
 'get 100': array([   5.28, 1497.  ,    8.  ]),
 'help relieve': array([  5.136, 162.   ,   1.   ]),
 '3d tomorrow': array([5., 5., 0.]),
 'feisty cat': array([ 5., 10.,  0.]),
 'compliment from': array([5., 6., 0.]),
 "that's correct": array([5., 6., 0.]),
 'at chicago': array([5., 5., 0.]),
 'disney songs': array([5., 5., 0.]),
 'soo exited': array([5., 7., 0.]),
 'baby picture': array([5., 5., 0.]),
 'australia next': array([5., 5., 0.]),
 'meester !': array([5., 5., 0.]),
 'tall ships': array([5., 6., 0.]),
 'day todayy': array([5., 6., 0.]),
 'snuggle under': array([5., 5., 0.]),
 '& laughter': array([5., 5., 0.]),
 't.t hank': array([5., 5., 0.]),
 ', organic': array([5., 6.

In [118]:
def get_unigram_sentiment_emoticon_lexicon_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_lexicon_unigram_dict.keys():
        vec_rep = emoticon_lexicon_unigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_emoticon_lexicon_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_lexicon_bigram_dict.keys():
        vec_rep = emoticon_lexicon_bigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

In [119]:
get_unigram_sentiment_emoticon_lexicon_vector("fury")

array([ 1.00000e+00, -6.20000e-02,  1.70000e+01,  1.90000e+01,
        3.84400e-03, -1.05400e+00, -1.17800e+00,  2.89000e+02,
        3.23000e+02,  3.61000e+02, -2.38328e-04,  6.53480e-02,
        7.30360e-02, -1.79180e+01, -2.00260e+01, -2.23820e+01,
        4.91300e+03,  5.49100e+03,  6.13700e+03,  6.85900e+03])

In [120]:
get_bigram_sentiment_emoticon_lexicon_vector("add everyone")

array([1.00000000e+00, 7.35200000e+00, 1.48500000e+03, 1.00000000e+00,
       5.40519040e+01, 1.09177200e+04, 7.35200000e+00, 2.20522500e+06,
       1.48500000e+03, 1.00000000e+00, 3.97389598e+02, 8.02670774e+04,
       5.40519040e+01, 1.62128142e+07, 1.09177200e+04, 7.35200000e+00,
       3.27475912e+09, 2.20522500e+06, 1.48500000e+03, 1.00000000e+00])

## Emoticon Sentiment Aff-Neg Lexicon

In [122]:
emoticon_afflex_unigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-AFFLEX-NEGLEX-unigrams.txt"

emoticon_afflex_bigrams_file_path = \
    wassa_home + \
    "lexicons/Emoticon-AFFLEX-NEGLEX-bigrams.txt"
    
emoticon_afflex_unigrams = dict()
emoticon_afflex_bigrams = dict()

def get_emoticon_afflex_unigram_dict():
    with open(emoticon_afflex_unigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_afflex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_afflex_unigrams

def get_emoticon_afflex_bigram_dict():
    with open(emoticon_afflex_bigrams_file_path) as emoticon_lexicon_file:
        for line in emoticon_lexicon_file:
            word_array = line.replace("\n", "").split("\t")
            emoticon_afflex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return emoticon_afflex_bigrams

In [123]:
emoticon_afflex_unigram_dict = get_emoticon_afflex_unigram_dict()

In [124]:
emoticon_afflex_bigram_dict = get_emoticon_afflex_bigram_dict()

In [125]:
def get_unigram_sentiment_emoticon_afflex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_afflex_unigram_dict.keys():
        vec_rep = emoticon_afflex_unigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_emoticon_afflex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in emoticon_afflex_bigram_dict.keys():
        vec_rep = emoticon_afflex_bigram_dict[word]
    
    return non_linear_factor.fit_transform([vec_rep])[0]

In [126]:
get_unigram_sentiment_emoticon_afflex_vector("fury")

array([1.00000e+00, 6.40000e-02, 1.40000e+01, 1.40000e+01, 4.09600e-03,
       8.96000e-01, 8.96000e-01, 1.96000e+02, 1.96000e+02, 1.96000e+02,
       2.62144e-04, 5.73440e-02, 5.73440e-02, 1.25440e+01, 1.25440e+01,
       1.25440e+01, 2.74400e+03, 2.74400e+03, 2.74400e+03, 2.74400e+03])

In [127]:
get_bigram_sentiment_emoticon_afflex_vector("pay vip")

array([1.00000000e+00, 7.33200000e+00, 1.44700000e+03, 0.00000000e+00,
       5.37582240e+01, 1.06094040e+04, 0.00000000e+00, 2.09380900e+06,
       0.00000000e+00, 0.00000000e+00, 3.94155298e+02, 7.77881501e+04,
       0.00000000e+00, 1.53518076e+07, 0.00000000e+00, 0.00000000e+00,
       3.02974162e+09, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

## Hashtag Sentiment Aff-Neg Lexicon

In [128]:
hashtag_affneglex_unigrams_file_path = \
    wassa_home + \
    "lexicons/HS-AFFLEX-NEGLEX-unigrams.txt"
hashtag_affneglex_bigrams_file_path = \
    wassa_home + \
    "lexicons/HS-AFFLEX-NEGLEX-bigrams.txt"
    
hashtag_affneglex_unigrams = dict()
hashtag_affneglex_bigrams = dict()

def get_hashtag_affneglex_unigram_dict():
    with open(hashtag_affneglex_unigrams_file_path) as hashtag_sent_lex_file:
        for line in hashtag_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            hashtag_affneglex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hashtag_affneglex_unigrams

def get_hashtag_affneglex_bigram_dict():
    with open(hashtag_affneglex_bigrams_file_path) as hashtag_sent_lex_file:
        for line in hashtag_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            hashtag_affneglex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])

    return hashtag_affneglex_bigrams

In [129]:
hashtag_affneglex_unigram_dict = get_hashtag_affneglex_unigram_dict()

In [130]:
hashtag_affneglex_bigram_dict = get_hashtag_affneglex_bigram_dict()

In [131]:
def get_unigram_sentiment_hashtag_affneglex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hashtag_affneglex_unigram_dict.keys():
        vec_rep = hashtag_affneglex_unigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

def get_bigram_sentiment_hashtag_affneglex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hashtag_affneglex_bigram_dict.keys():
        vec_rep = hashtag_affneglex_bigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]

In [132]:
get_unigram_sentiment_hashtag_affneglex_vector("#great")

array([1.00000000e+00, 1.06610000e+01, 3.00520000e+04, 0.00000000e+00,
       1.13656921e+02, 3.20384372e+05, 0.00000000e+00, 9.03122704e+08,
       0.00000000e+00, 0.00000000e+00, 1.21169643e+03, 3.41561779e+06,
       0.00000000e+00, 9.62819115e+09, 0.00000000e+00, 0.00000000e+00,
       2.71406435e+13, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [133]:
get_bigram_sentiment_hashtag_affneglex_vector("#good luck")

array([1.00000000e+00, 5.92000000e+00, 2.88000000e+02, 0.00000000e+00,
       3.50464000e+01, 1.70496000e+03, 0.00000000e+00, 8.29440000e+04,
       0.00000000e+00, 0.00000000e+00, 2.07474688e+02, 1.00933632e+04,
       0.00000000e+00, 4.91028480e+05, 0.00000000e+00, 0.00000000e+00,
       2.38878720e+07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

## Hashtag Sentiment Lexicon

In [134]:
hash_sent_lex_unigrams_file_path = \
    wassa_home + \
    "lexicons/HS-unigrams.txt"
hash_sent_lex_bigrams_file_path = \
    wassa_home + \
    "lexicons/HS-bigrams.txt"

def get_hash_sent_lex_unigram_dict():
    
    hash_sent_lex_unigrams = dict()
    with open(hash_sent_lex_unigrams_file_path) as hash_sent_lex_file:
        for line in hash_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if clean_str(word_array[0]):
                hash_sent_lex_unigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hash_sent_lex_unigrams

def get_hash_sent_lex_bigram_dict():

    hash_sent_lex_bigrams = dict()
    with open(hash_sent_lex_bigrams_file_path) as hash_sent_lex_file:
        for line in hash_sent_lex_file:
            word_array = line.replace("\n", "").split("\t")
            if clean_str(word_array[0]):
                hash_sent_lex_bigrams[word_array[0]] = np.array([float(val) for val in word_array[1:]])
    
    return hash_sent_lex_bigrams

In [135]:
hash_sent_lex_unigram_dict = get_hash_sent_lex_unigram_dict()

In [136]:
hash_sent_lex_bigram_dict = get_hash_sent_lex_bigram_dict()

In [137]:
def get_unigram_sentiment_hash_sent_lex_vector(word):
    
    vec_rep = np.zeros(3)
    if word in hash_sent_lex_unigram_dict.keys():
        vec_rep = hash_sent_lex_unigram_dict[word]
        
    return non_linear_factor.fit_transform([vec_rep])[0]


def get_bigram_sentiment_hash_sent_lex_vector(word):

    vec_rep = np.zeros(3)
    if word in hash_sent_lex_bigram_dict.keys():
        vec_rep = hash_sent_lex_bigram_dict[word]
            
    return non_linear_factor.fit_transform([vec_rep])[0]

In [138]:
get_unigram_sentiment_hash_sent_lex_vector("#fabulous")

array([1.00000000e+00, 7.52600000e+00, 2.30100000e+03, 2.00000000e+00,
       5.66406760e+01, 1.73173260e+04, 1.50520000e+01, 5.29460100e+06,
       4.60200000e+03, 4.00000000e+00, 4.26277728e+02, 1.30330195e+05,
       1.13281352e+02, 3.98471671e+07, 3.46346520e+04, 3.01040000e+01,
       1.21828769e+10, 1.05892020e+07, 9.20400000e+03, 8.00000000e+00])

In [139]:
get_bigram_sentiment_hash_sent_lex_vector(". #perfection")

array([1.00000000e+00, 8.88800000e+00, 4.49300000e+03, 1.00000000e+00,
       7.89965440e+01, 3.99337840e+04, 8.88800000e+00, 2.01870490e+07,
       4.49300000e+03, 1.00000000e+00, 7.02121283e+02, 3.54931472e+05,
       7.89965440e+01, 1.79422492e+08, 3.99337840e+04, 8.88800000e+00,
       9.07004112e+10, 2.01870490e+07, 4.49300000e+03, 1.00000000e+00])

## Depeche Mood (There is DepecheMood V2, try it)

In [147]:
depeche_mood_file_path = \
    wassa_home + \
    "lexicons/DepecheMood_V1.0/DepecheMood_normfreq2.txt"

In [148]:
def get_depeche_vector_dict():
    depeche_vector_dict = dict()
    with open(depeche_mood_file_path) as depeche_mood_file:
        for line in depeche_mood_file:
            word_array = line.replace("\n", "").split("\t")
            depeche_vector_dict[word_array[0].split("#")[0]] = np.array([float(val) for val in word_array[1:]])
    
    return depeche_vector_dict

In [149]:
depeche_vector_dict = get_depeche_vector_dict()

In [150]:
def get_depeche_mood_vector(word):
    
    vec_rep = np.zeros(8)
    if word in depeche_vector_dict.keys():
        vec_rep = np.array(depeche_vector_dict[word])

    return non_linear_factor.fit_transform([vec_rep])[0]

In [151]:
get_depeche_mood_vector("120th")

array([1.        , 0.        , 0.5643932 , 0.        , 0.        ,
       0.        , 0.1727759 , 0.2628309 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.31853969, 0.        , 0.        ,
       0.        , 0.09751354, 0.14833997, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02985151,
       0.04541085, 0.        , 0.06908008, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

# Reading & Vectorizing Data

In [ ]:
#     print(embedding_features.shape)
    
#     lexicon_features = get_unigram_embedding(word, embedding_info[0], unigram_feature_string)
#     poly_lexicon_features = non_linear_factor.fit_transform([lexicon_features])[0]
#     print(poly_lexicon_features.shape)

#     final_features = np.concatenate((embedding_features, poly_lexicon_features))
#     print(final_features.shape)

In [152]:
def is_active_vector_method(string):
    return int(string)

def learn_unigram_word_embedding(word):
    
    word_feature_embedding_dict = dict()
    
    '''Pre-trained Word embeddings'''
    index = 0
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model, w2v_dimensions)

    index = 1
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_1, w2v_dimensions_1)

    index = 2
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_2, w2v_dimensions_2)

    index = 3
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_3, w2v_dimensions_3)

    index = 4
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_4, w2v_dimensions_4)

    index = 5
    word_feature_embedding_dict[index] = get_word2vec_embedding(word, wv_model_5, w2v_dimensions_5)

    '''NRC Emotion Intensity Lexicon'''
    index = 6
    word_feature_embedding_dict[index] = get_emo_int_vector(word)

    '''WordNet'''
    index = 7
    word_feature_embedding_dict[index] = get_sentiwordnetscore(word)

    '''NRC Sentiment Lexica'''
    index = 8
    word_feature_embedding_dict[index] = get_sentiment_emotion_feature(word)

    index = 9
    word_feature_embedding_dict[index] = get_unigram_sentiment_emoticon_lexicon_vector(word)

    index = 10
    word_feature_embedding_dict[index] = get_unigram_sentiment_emoticon_afflex_vector(word)

    '''NRC Hashtag Lexica'''
    index = 11
    word_feature_embedding_dict[index] = get_hashtag_emotion_vector(word)

    index = 12
    word_feature_embedding_dict[index] = get_unigram_sentiment_hash_sent_lex_vector(word)

    index = 13
    word_feature_embedding_dict[index] = get_unigram_sentiment_hashtag_affneglex_vector(word)

    '''Emoji Polarities'''
    index = 14
    word_feature_embedding_dict[index] = get_emoji_intensity(word)
    
    '''Depeche Mood'''
    index = 15
    word_feature_embedding_dict[index] = get_depeche_mood_vector(word)

    return word_feature_embedding_dict


def learn_bigram_word_embedding(word):
    
    word_feature_embedding_dict = dict()
    
    '''NRC Sentiment Lexica'''

    index = 0
    word_feature_embedding_dict[index] = get_bigram_sentiment_emoticon_lexicon_vector(word)

    index = 1
    word_feature_embedding_dict[index] = get_bigram_sentiment_emoticon_afflex_vector(word)

    '''NRC Hashtag Lexica'''
    index = 2
    word_feature_embedding_dict[index] = get_bigram_sentiment_hash_sent_lex_vector(word)

    index = 3
    word_feature_embedding_dict[index] = get_bigram_sentiment_hashtag_affneglex_vector(word)

    return word_feature_embedding_dict

In [256]:
get_word2vec_embedding(word, wv_model_5, w2v_dimensions_5)

array([ 4.6974e-02, -4.1924e-02,  2.1134e-01,  4.1058e-02, -6.3082e-01,
       -4.1387e-02, -2.9409e-01, -7.3062e-02, -4.1351e-01,  2.1914e+00,
       -1.7096e-01,  3.6084e-01, -1.0269e-01,  5.8762e-02, -4.7663e-01,
        2.3331e-01, -4.4095e-01,  8.9501e-01,  1.7049e-01,  3.7093e-01,
       -4.8250e-01, -2.3186e-01, -6.3895e-03,  2.8126e-01, -1.0865e-01,
        4.4211e-01, -1.4450e-02, -4.4916e-01,  2.2539e-01,  1.0822e-01,
       -2.6223e-01, -1.8789e-01,  2.9297e-01,  2.3474e-01, -3.8838e-01,
       -1.9569e-02, -2.2589e-01,  2.1374e-03, -9.7055e-02,  2.9788e-01,
       -1.3896e-02,  5.3449e-01, -7.7740e-02,  7.3947e-02, -4.9664e-01,
        1.6229e-01, -3.2101e-01,  3.5513e-01,  4.0022e-01, -2.4462e-01,
       -4.0997e-01, -2.2242e-01, -2.6443e-02, -4.3470e-01,  8.4170e-02,
       -3.9290e-01, -1.2373e-01, -3.4772e-01,  5.6221e-01,  9.5894e-02,
       -3.2456e-01, -5.7979e-01, -4.6646e-01, -1.5448e-01,  4.9639e-01,
        4.1833e-01,  5.4878e-01,  1.7582e-01,  7.6796e-01, -2.15

In [153]:
def get_unigram_embedding(word, word_embedding_dict, bin_string):
    
    word_feature_embedding_dict = word_embedding_dict[word]
    final_embedding = np.array([])
    
    for i in range(16):
        if is_active_vector_method(bin_string[i]):
            final_embedding = np.append(final_embedding, word_feature_embedding_dict[i])
    
    return final_embedding

def get_bigram_embedding(bigram, word_embedding_dict, bin_string):
    
    word_feature_embedding_dict = word_embedding_dict[word]
    final_embedding = np.array([])
    
    for i in range(4):
        if is_active_vector_method(bin_string[i]):
            final_embedding = np.append(final_embedding, word_feature_embedding_dict[i])
    
    return final_embedding

In [154]:
unigram_feature_string = "1111111111111111"
bigram_feature_string = "1111"

In [166]:
training_tweets = read_training_data(training_data_file_path)
dev_tweets = read_training_data(dev_set_path)

score_train = list()
tweet_train = list()
for tweet in training_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))

for tweet in dev_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))
    
print(len(score_train))
score_train = np.asarray(score_train)

941


In [167]:
raw_test_tweets = read_training_data_verbatim(test_data_file_path)
test_tweets = read_training_data(test_data_file_path)

tweet_test_raw = list()
tweet_test = list()
y_gold = list()

for tweet in raw_test_tweets:
    tweet_test_raw.append(tweet.text)

for tweet in test_tweets:
    tweet_test.append(tweet.text)
    y_gold.append(float(tweet.intensity))
    
print(len(y_gold))

760


In [168]:
def build_word_embeddings(tweets):
    
    max_tweet_length = -1
    word_embedding_dict = dict()

    for tweet in tweets:
        if len(tweet) > max_tweet_length:
            max_tweet_length = len(tweet)

        for token in tweet:
            if token not in word_embedding_dict.keys():
                word_embedding_dict[token] = learn_unigram_word_embedding(token)
                
    return word_embedding_dict, max_tweet_length

In [192]:
all_tweets = tweet_train + tweet_test
embedding_info = build_word_embeddings(all_tweets)

In [193]:
# Save vectors
with open(word_embeddings_path, 'wb') as word_embeddings_file:
     pickle.dump(embedding_info, word_embeddings_file)

In [171]:
browser_notify("Persisted to disk")

In [194]:
# Restore vectors
with open(word_embeddings_path, 'rb') as word_embeddings_file:
    embedding_info = pickle.load(word_embeddings_file)

In [195]:
embeddings_index = embedding_info[0]
MAX_SEQUENCE_LENGTH = embedding_info[1]
MAX_NB_WORDS = 20000
EMBEDDING_DIM = len(get_unigram_embedding("glad", embedding_info[0], unigram_feature_string))
print(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)

37 2071


In [196]:
word_indices = dict()
current_index = 1

In [197]:
def sequence_tweets(tweets):
    global current_index
    vectors = list()
    for tweet in tweets:        
        vector = list()
        for word in tweet:
            word_index = None
            
            if word in word_indices:
                word_index = word_indices[word]
            else:
                word_index = current_index
                current_index += 1
                word_indices[word] = word_index
            
            vector.append(word_index)
        
        vectors.append(vector)

    return vectors

In [198]:
x_train = sequence_tweets(tweet_train)
x_test = sequence_tweets(tweet_test)

In [199]:
len(word_indices)

4715

In [200]:
display(tweet_train)

[['fu',
  'k',
  '!',
  'heck',
  '!',
  'moved',
  'fridge',
  '!',
  'knock',
  'landlord',
  'door',
  'angry',
  'mad'],
 ['indian',
  'uber',
  'driver',
  'called',
  'someone',
  'n',
  'word',
  'moving',
  'vehicle',
  'would',
  'jumped',
  'disgusted'],
 ['asked',
  'parcel',
  'delivered',
  'pick',
  'store',
  'address',
  'fuming',
  'poorcustomerservice'],
 ['ef',
  'whichever',
  'butt',
  'wipe',
  'pulled',
  'fire',
  'alarm',
  'davis',
  'bc',
  'sound',
  'asleep',
  'pissed',
  'angry',
  'upset',
  'tired',
  'sad',
  'tired',
  'hangry'],
 ['join',
  'put',
  'phone',
  'talk',
  'rude',
  'taking',
  'money',
  'acc',
  'willynilly',
  '!',
  'fuming'],
 ['blood', 'boiling'],
 ['still',
  'got',
  'whole',
  'season',
  'wentworth',
  'watch',
  'stupid',
  'cunt',
  'work',
  'ruins',
  'us',
  '😭',
  '😭',
  'raging',
  'oldcunt'],
 ['tracking',
  'show',
  'equipment',
  'delivered',
  '?',
  'service',
  'suddenly',
  'delayed',
  '?',
  'already',
  'week

In [201]:
word_embedding_matrix = list()
word_embedding_matrix.append(np.zeros(EMBEDDING_DIM))

for word in sorted(word_indices, key=word_indices.get):
    embedding_features = get_unigram_embedding(word, embedding_info[0], unigram_feature_string)    
    word_embedding_matrix.append(embedding_features)

word_embedding_matrix = np.asarray(word_embedding_matrix, dtype='f')

In [202]:
word_embedding_matrix.shape

(4716, 2071)

In [203]:
word_embedding_matrix = scale(word_embedding_matrix)

C:\Users\COMMANDER\.conda\envs\gputest\lib\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\COMMANDER\.conda\envs\gputest\lib\site-packages\sklearn\preprocessing\_data.py:191: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [204]:
browser_notify("Vectorization Done")

# Recurrent Neural Network Implementation in Keras

In [205]:
pre_padding = 6

x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

x_train = sequence.pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH + pre_padding, padding="pre")
x_test = sequence.pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH + pre_padding, padding="pre")

In [206]:
len(x_train), len(x_test), len(x_train[0])

(941, 760, 43)

In [228]:
embed_1 = Embedding(len(word_indices) + 1, EMBEDDING_DIM, weights=[word_embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH + pre_padding, trainable=True)

conv_1 = Conv1D(128, 3, activation='relu', name='conv1')
conv_2 = Conv1D(128, 3, activation='relu', name='conv2')
conv_3 = Conv1D(256, 3, activation='relu', name='conv3')
conv_4 = Conv1D(256, 3, activation='relu', name='conv4')
conv_5 = Conv1D(256, 3, activation='relu', name='conv5')
conv_6 = Conv1D(1024, 3, activation='relu', name='conv6')
conv_7 = Conv1D(1024, 3, activation='relu', name='conv7')
conv_8 = Conv1D(1024, 3, activation='relu', name='conv8')

pool_1 = AveragePooling1D(pool_size=3, strides=2, name='pool1')
pool_2 = AveragePooling1D(pool_size=3, strides=2, name='pool2')
pool_3 = MaxPooling1D(pool_size=3, strides=2, name='pool3')
pool_4 = MaxPooling1D(pool_size=3, strides=2, name='pool4')

lstm_1 = LSTM(256, dropout=0.2, recurrent_dropout=0.2, name='lstm1', return_sequences=True)
lstm_2 = LSTM(128, dropout=0.2, recurrent_dropout=0.2, name='lstm2', return_sequences=True)
lstm_3 = LSTM(64, dropout=0.2, recurrent_dropout=0.2, name='lstm3')

gru_1 = GRU(256, dropout=0.25, recurrent_dropout=0.25, name='gru1', return_sequences=True)
gru_2 = GRU(256, dropout=0.25, recurrent_dropout=0.25, name='gru2', return_sequences=True)
gru_3 = GRU(256, dropout=0.25, recurrent_dropout=0.25, name='gru3')

bi_lstm_1 = Bidirectional(lstm_1, name='bilstm1')
bi_lstm_2 = Bidirectional(lstm_2, name='bilstm2')
bi_lstm_3 = Bidirectional(lstm_3, name='bilstm3')

dense_1 = Dense(20000, activation='relu', name='dense1')
dense_2 = Dense(1, activation='sigmoid', name='dense2')

drop_1 = Dropout(0.5, name='drop1')
drop_2 = Dropout(0.5, name='drop2')

In [208]:
def get_rnn_model():
    
    model = Sequential()
    
    model.add(embed_1)
    
    model.add(conv_1)
    model.add(conv_2)
#     model.add(pool_1)
    
#     model.add(conv_3)
#     model.add(conv_4)
#     model.add(pool_2)
    
#     model.add(conv_5)
    
    model.add(lstm_1)
    model.add(lstm_2)
    model.add(lstm_3)
    
    model.add(dense_1)
#     model.add(drop_1)
    
    model.add(dense_2)

    model.compile(loss='mean_squared_error', optimizer="adam")
    
    return model

In [249]:
nn_model = KerasRegressor(build_fn=get_rnn_model, epochs=50, batch_size=64, verbose=1)

score_train = np.asarray(score_train)

# ml_model = AdaBoostRegressor(base_estimator=nn_model)

nn_model.fit(x_train, score_train , epochs=100, batch_size=64, verbose=1)
y_pred = nn_model.predict(x_test)

C:\Users\COMMANDER\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
941/941 [==============================] - 3s 3ms/step - loss: 0.0029
Epoch 2/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0036
Epoch 3/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0029
Epoch 4/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 5/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 6/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0023
Epoch 7/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0022
Epoch 8/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0020
Epoch 9/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0018
Epoch 10/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0017
Epoch 11/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0016
Epoch 12/100
941/941 [==============================] - 2s 2ms/step - loss: 0.0017
Epoch 13/100


941/941 [==============================] - 2s 2ms/step - loss: 3.2007e-04
Epoch 97/100
941/941 [==============================] - 2s 2ms/step - loss: 3.1212e-04
Epoch 98/100
941/941 [==============================] - 2s 2ms/step - loss: 3.4495e-04
Epoch 99/100
941/941 [==============================] - 2s 2ms/step - loss: 3.3016e-04
Epoch 100/100
760/760 [==============================] - 0s 526us/step


In [230]:
browser_notify("NN Training Done")

In [250]:
y_pred = np.reshape(y_pred, len(y_pred))

In [251]:
print(emotion)
print(pearsonr(y_pred, y_gold))
print(spearmanr(y_pred, y_gold))

anger
(0.6587386062976041, 9.514470039350388e-96)
SpearmanrResult(correlation=0.6457990581411007, pvalue=6.66072067009075e-91)


In [233]:
# with open(predictions_file_path, 'w') as predictions_file:
#     for i in range(len(y_pred)):        
#         predictions_file.write(
#             str(raw_test_tweets[i].id) + "\t" + \
#             raw_test_tweets[i].text + "\t" + \
#             raw_test_tweets[i].emotion + "\t" + \
#             str(y_pred[i]) + "\n"
#         )

In [234]:
browser_notify("Neural Net Trained")

# Analysis

In [235]:
def get_predictions_list(y_pred, y_gold):
    predictions = list()
    for i in range(len(y_gold)):
        prediction = {
            "raw_tweet": tweet_test_raw[i], 
            "cleaned_tweet": tweet_test[i], 
            "prediction": y_pred[i], 
            "actual": y_gold[i], 
            "diff": (y_gold[i] - y_pred[i])
        }
        predictions.append(prediction)
        
    return predictions

In [236]:
predictions = get_predictions_list(y_pred, y_gold)

In [237]:
pred_df = DataFrame(predictions)
pred_df = pred_df[["raw_tweet", "cleaned_tweet", "prediction", "actual", "diff"]]
pred_df = pred_df.sort_values(by=['diff'], ascending=[True])

In [238]:
display(pred_df)

,raw_tweet,cleaned_tweet,prediction,actual,diff
165,ethan was with someone in the car when he did ...,"[ethan, someone, car, revenge, grayson]",0.680146,0.292,-0.388146
656,@Uncle_Brown22 yeah dude! Appreciate the #supp...,"[yeah, dude, !, appreciate, support, !, notori...",0.557101,0.208,-0.349101
372,my dogs making the most RIDICULOUS sounds righ...,"[dogs, making, ridiculous, sounds, right, like...",0.649427,0.333,-0.316427
116,@WinchesterGank If only. Lol I just wish once ...,"[lol, wish, understood, crazy, rabid, fans, &,...",0.663255,0.356,-0.307255
604,Men in rage strike those that wish them best. ...,"[men, rage, strike, wish, best, emotions, nega...",0.591954,0.292,-0.299954
...,...,...,...,...,...
338,@AmericanAir I am irate at you check-in system...,"[irate, check, system, tried, checkin, day, wo...",0.386169,0.729,0.342831
450,@EE your website is making me feel violent rag...,"[website, making, feel, violent, rage, upgrade...",0.531539,0.934,0.402461
174,Dear hipster behind me at the game I am findin...,"[dear, hipster, behind, game, finding, hard, p...",0.435573,0.854,0.418427
191,My irritation level is at an all time high tod...,"[irritation, level, time, high, today, 🙃]",0.298680,0.750,0.451320


In [240]:
pred_df.to_csv('pred.csv')

In [239]:
with open(debug_file_path, 'w') as debug_file:
    pred_df.to_csv(debug_file, sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/COMMANDER/Downloads/dataset/test-set/debug/anger.tsv'